In [1]:
# City of Chicago 
## Summary:
### Average Sale price in the city of Chicago is about 1.5 M.
### Chicago likes mid size house with 8 rooms, 3 bedrooms, 2 baths( NOT 2.5 baths)!!!
### Size of parking lot did not have a huge impact on  house sales.

In [4]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import sem

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# File to Load (Remember to Change These)
chi_data = "data/lm9new_AllEvents_City_Chicago_Property_data1.csv"


# Read the Data
chi_df = pd.read_csv(chi_data)



In [5]:
chi_df.head()

,IdentifierObPropId,IdentifierFips,IdentifierApn,IdentifierApnOrig,IdentifierAttomId,LotDepth,LotFrontage,LotLotsize1,LotLotsize2,LotPooltype,...,AssessmentCalculationsCalclandvalue,AssessmentCalculationsCalcttlind,AssessmentCalculationsCalcttlvalue,AssessmentCalculationsCalcvaluepersizeunit,AssessmentMarketMktimprvalue,AssessmentMarketMktlandvalue,AssessmentMarketMktttlvalue,AssessmentTaxTaxamt,AssessmentTaxTaxpersizeunit,AssessmentTaxTaxyear
0,17590035317031,17031,17343060150000,17343060150000,175900353,0,0,0.0737,3211,NONE,...,0,MARKET VALUE,0,0.0,0,0,0,0.0,0.0,0
1,17489261517031,17031,25242100010000,25242100010000,174892615,0,0,0.0636,2770,NONE,...,0,MARKET VALUE,0,0.0,0,0,0,133.0,0.0,2017
2,15850885717031,17031,17221090260000,17221090260000,158508857,0,0,0.0687,2992,NONE,...,0,MARKET VALUE,0,0.0,0,0,0,0.0,0.0,0
3,17519655817031,17031,17221060540000,17221060540000,175196558,0,0,0.0302,1317,NONE,...,0,MARKET VALUE,0,0.0,0,0,0,107.0,0.0,2017
4,17589563117031,17031,17271150250000,17271150250000,175895631,0,0,0.2090,9103,NONE,...,0,MARKET VALUE,0,0.0,0,0,0,0.0,0.0,0


In [6]:
# new columns
chi_cleaned = chi_df[['AddressOneLine','LotLotsize2','LocationLongitude',\
'LocationLatitude','UtilitiesHeatingtype',\
'BuildingSizeLivingsize',\
'BuildingRoomsBathstotal','BuildingRoomsBeds','BuildingRoomsRoomsTotal',\
'SaleAmountSaleamt','AssessmentTaxTaxamt','SummaryProptype']]

# fill blank values with 0
chi_cleaned.fillna(0, inplace=True)

# clean data results
chi_cleaned = chi_cleaned[chi_cleaned['SaleAmountSaleamt'] > 0]
chi_cleaned = chi_cleaned[chi_cleaned['AssessmentTaxTaxamt'] > 0]

chi_cleaned = chi_cleaned[chi_cleaned['SummaryProptype'] != 'VACANT LAND (NEC)']
chi_cleaned = chi_cleaned[chi_cleaned['SummaryProptype'] != 'TRANSPORT FACILITY']
chi_cleaned = chi_cleaned[chi_cleaned['SummaryProptype'] != 'PUBLIC (NEC)']
chi_cleaned = chi_cleaned[chi_cleaned['SummaryProptype'] != 'STORE BUILDING']
chi_cleaned = chi_cleaned[chi_cleaned['SummaryProptype'] != 'COMMON AREA']
chi_cleaned = chi_cleaned[chi_cleaned['SummaryProptype'] != 'RESIDENTIAL ACREAGE']
chi_cleaned = chi_cleaned[chi_cleaned['SummaryProptype'] != 'PARKING STRUCTURE']
chi_cleaned = chi_cleaned[chi_cleaned['SummaryProptype'] != 'INDUSTRIAL (NEC)']
chi_cleaned = chi_cleaned[chi_cleaned['SummaryProptype'] != 'MISC IMPROVEMENTS']

chi_cleaned_renamed_df  = chi_cleaned.rename(columns = {"SummaryYearbuilt": "Year Built", "SaleAmountSaleamt" : "Sale Amount"})

# reset index
chi_cleaned_renamed_df.reset_index(drop=True)

# display
chi_cleaned_renamed_df

,AddressOneLine,LotLotsize2,LocationLongitude,LocationLatitude,UtilitiesHeatingtype,BuildingSizeLivingsize,BuildingRoomsBathstotal,BuildingRoomsBeds,BuildingRoomsRoomsTotal,Sale Amount,AssessmentTaxTaxamt,SummaryProptype
34,"2640 E MARTHA PL, CHICAGO, IL 60633",2846,-87.553701,41.630573,0,1405,2.0,3,0,35000,4784.0,CONDOMINIUM
36,"57 W 146TH ST, CHICAGO, IL 60827",4687,-87.623593,41.630011,0,1151,1.0,4,0,66500,5716.0,SFR
44,"14535 S HOXIE AVE, CHICAGO, IL 60633",5000,-87.560227,41.630466,0,1156,1.0,3,0,47500,4932.0,SFR
45,"2628 E MARTHA PL, CHICAGO,IL 60633",2079,-87.552900,41.630650,0,1424,2.0,3,0,150000,4810.0,CONDOMINIUM
46,"2628 E MARTHA PL, CHICAGO,IL 60633",2079,-87.552900,41.630650,0,1424,2.0,3,0,150000,4810.0,CONDOMINIUM
48,"14530 S NORMAL AVE, CHICAGO, IL 60827",5000,-87.634345,41.630240,0,1082,1.0,2,0,55500,161.0,SFR
52,"14531 S DEARBORN ST, CHICAGO, IL 60827",4718,-87.623037,41.630333,0,991,1.0,3,0,13000,4014.0,SFR
53,"119 E 146TH ST, CHICAGO,IL 60827",5323,-87.618619,41.630171,0,1172,1.0,3,0,123000,3624.0,SFR
64,"14521 S MARQUETTE AVE, CHICAGO, IL 60633",5360,-87.556517,41.631029,0,1224,2.0,3,0,140000,6083.0,SFR
68,"14535 S BENSLEY AVE, CHICAGO, IL 60633",5040,-87.562637,41.630470,0,1252,2.0,3,0,114000,3878.0,SFR
